In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

## Define dataset and variables

In [2]:
case = "fanv3_debugging"
path = "/glade/scratch/jinmuluo/archive/fanv3_debugging/lnd/hist/" 
start_date = date(2000, 1, 1)
end_date = date(2000, 12, 1)
delta = end_date - start_date

In [3]:
MID_TERM_FAN = ["FERT_NO3_TO_SOIL", "MANURE_NO3_TO_SOIL", 'area', 'landfrac']
RUNOFF = ["MANURE_NITRATE_RUNOFF", 'FERT_NITRATE_RUNOFF', 
            "FERT_NITRATE_TO_SOIL", "MANURE_NITRATE_TO_SOIL", 'area','landfrac']
GAS = ["N2O_DENITRIFY_TOTAL", "N2O_NITRIFY_TOTAL", "NOx_DENITRIFY_TOTAL", "NOx_NITRIFY_TOTAL", 
            "N2_TOTAL", "area", 'landfrac']
POOLS = ["NITRATE_TOTN", "NITRATE_G1", "NITRATE_G2", "NITRATE_G3", "NITRATE_S0", "NITRATE_S1", "NITRATE_S2",
            "NITRATE_S3", "NITRATE_F1", "NITRATE_F2",  "NITRATE_F3",  "NITRATE_F4", 'area', 'landfrac']

## (1) Nitrification flux from fanv2

In [4]:
fanCLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h2" + "." + month +".nc")
# print(fanCLM)

In [5]:
def preprocess(ds, fields = MID_TERM_FAN ):
    return(ds[fields])

dsCLM = xr.open_mfdataset(fanCLM, decode_times=True, preprocess=preprocess)

In [6]:
ds_fert = dsCLM["FERT_NO3_TO_SOIL"] * dsCLM['area']*dsCLM['landfrac']*1e6
ds_manure = dsCLM["MANURE_NO3_TO_SOIL"] * dsCLM['area']*dsCLM['landfrac']*1e6
ds_fert = ds_fert.sum(dim=['lat', 'lon'])
ds_manure = ds_manure.sum(dim=['lat', 'lon'])

fert = 0.0
manure = 0.0
fanv2_flux = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds_fert.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    
    fert = fert +  ds_fert[i].values * day * 24 * 3600
    manure = manure +  ds_manure[i].values * day * 24 * 3600

fert = fert * 1e-12
manure = manure * 1e-12
fanv2_flux = fert + manure

print("fertilization flux:", fert, "Tg")
print("manure flux:", manure, "Tg")
print(fanv2_flux, "Tg")

fertilization flux: 20.680319105845314 Tg
manure flux: 12.4287913323 Tg
33.109110438145315 Tg


## (2) NOx, N2O and N2 emission 

In [7]:
fanCLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h0" + "." + month +".nc")

In [8]:
def preprocess(ds, fields = GAS):
    return(ds[fields])

dsCLM = xr.open_mfdataset(fanCLM, decode_times=True, preprocess=preprocess)
# dsCLM

In [9]:
ds_n2o_nit = dsCLM["N2O_NITRIFY_TOTAL"] * dsCLM['area']*dsCLM['landfrac']*1e6
ds_n2o_denit = dsCLM["N2O_DENITRIFY_TOTAL"]* dsCLM['area']*dsCLM['landfrac']*1e6
ds_nox_nit = dsCLM["NOx_NITRIFY_TOTAL"] * dsCLM['area']*dsCLM['landfrac']*1e6
ds_nox_denit = dsCLM["NOx_DENITRIFY_TOTAL"] * dsCLM['area']*dsCLM['landfrac']*1e6
ds_n2 = dsCLM["N2_TOTAL"] * dsCLM['area']*dsCLM['landfrac']*1e6

ds_n2o_nit = ds_n2o_nit.fillna(0).sum(dim=['lat', 'lon'])
ds_n2o_denit = ds_n2o_denit.fillna(0).sum(dim=['lat', 'lon'])
ds_nox_nit = ds_nox_nit.fillna(0).sum(dim=['lat', 'lon'])
ds_nox_denit = ds_nox_denit.fillna(0).sum(dim=['lat', 'lon'])
ds_n2 = ds_n2.sum(dim=['lat', 'lon'])

n2o_nit_total = 0.0
n2o_denit_total = 0.0
nox_nit_total = 0.0
nox_denit_total = 0.0
n2_total = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds_n2.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    n2o_nit_total = n2o_nit_total + ds_n2o_nit[i].values * day * 24 * 3600
    n2o_denit_total = n2o_denit_total + ds_n2o_denit[i].values * day * 24 * 3600
    nox_nit_total = nox_nit_total + ds_nox_nit[i].values * day * 24 * 3600
    nox_denit_total = nox_denit_total + ds_nox_denit[i].values * day * 24 * 3600
    n2_total = n2_total + ds_n2[i].values * day * 24 * 3600
    
n2o_nit_total = n2o_nit_total * 1e-12
n2o_denit_total = n2o_denit_total * 1e-12
nox_nit_total = nox_nit_total * 1e-12
nox_denit_total = nox_denit_total * 1e-12
n2_total = n2_total * 1e-12

print('N2O from nitrification:', n2o_nit_total, "Tg")
print('N2O from denitrification:', n2o_denit_total, "Tg")
print('NOx from nitrification:', nox_nit_total, "Tg")
print('NOx from denitrification:', nox_denit_total, "Tg")
print('N2', n2_total, "Tg")

N2O from nitrification: 0.4453174612265625 Tg
N2O from denitrification: 0.000619346780557251 Tg
NOx from nitrification: 1.0596489886125 Tg
NOx from denitrification: 0.0007439273716278077 Tg
N2 0.010447747922460938 Tg


## (3) Diffusion and Runoff

In [10]:
fanCLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h1" + "." + month +".nc")

In [11]:
def preprocess(ds, fields = RUNOFF):
    return(ds[fields])

dsCLM = xr.open_mfdataset(fanCLM, decode_times=True, preprocess=preprocess)
#dsCLM

In [12]:
ds_soil_fert = dsCLM["FERT_NITRATE_TO_SOIL"] * dsCLM['area']*dsCLM['landfrac']*1e6
ds_soil_manure = dsCLM["MANURE_NITRATE_TO_SOIL"] * dsCLM['area']*dsCLM['landfrac']*1e6
ds_runoff_fert = dsCLM["FERT_NITRATE_RUNOFF"] * dsCLM['area']*dsCLM['landfrac']*1e6
ds_runoff_manure = dsCLM["MANURE_NITRATE_RUNOFF"] * dsCLM['area']*dsCLM['landfrac']*1e6

ds_soil_fert = ds_soil_fert.fillna(0).sum(dim=['lat', 'lon'])
ds_soil_manure = ds_soil_manure.fillna(0).sum(dim=['lat', 'lon'])
ds_runoff_fert = ds_runoff_fert.fillna(0).sum(dim=['lat', 'lon'])
ds_runoff_manure = ds_runoff_manure.fillna(0).sum(dim=['lat', 'lon'])

soil_fert = 0.0
soil_manure = 0.0
runoff_fert = 0.0
runoff_manure = 0.0

# unit transfer form gN/s to Tg/year
for i in range(len(ds_runoff_fert.time)):
    t= start_date + relativedelta(months=i)
    t2 = start_date + relativedelta(months=i+1)
    day = (t2-t).days
    
    soil_fert = soil_fert + ds_soil_fert[i].values * day * 24 * 3600
    soil_manure = soil_manure + ds_soil_manure[i].values * day * 24 * 3600
    runoff_fert = runoff_fert + ds_runoff_fert[i].values * day * 24 * 3600
    runoff_manure = runoff_manure+ ds_runoff_manure[i].values * day * 24 * 3600
    
soil_fert = soil_fert * 1e-12
soil_manure = soil_manure * 1e-12
runoff_fert = runoff_fert * 1e-12
runoff_manure = runoff_manure * 1e-12

print("diffusion to soil (fertilization)", soil_fert, "Tg")
print("diffusion to soil (manure)", soil_manure, "Tg")
print("surface runoff (fertilization)", runoff_fert, "Tg")
print("surface runoff (manure)", runoff_manure, "Tg")

diffusion to soil (fertilization) 17.982250123443748 Tg
diffusion to soil (manure) 9.948862799325 Tg
surface runoff (fertilization) 1.9538311623328124 Tg
surface runoff (manure) 1.170666521690625 Tg


## (4) Nitrite pools

In [13]:
fanCLM = []
for i in range(delta.days//30 + 1):
    if start_date.month != 1:
        date_term = date(start_date.year, start_date.month - 1, start_date.day)
    else:
        date_term = start_date
    month = date_term + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    fanCLM.append(path + case + ".clm2." + "h3" + "." + month +".nc")

In [14]:
dsCLM_end = xr.open_dataset(fanCLM[-1])
dsCLM_begin = xr.open_dataset(fanCLM[0])

In [15]:
pools_end = 0.0
pools_begin = 0.0

for pool in POOLS:
    if pool != 'area' and pool!= 'landfrac' and pool!= 'NITRATE_TOTN':
        pools_end = pools_end + (dsCLM_end[pool].fillna(0) * dsCLM_end["area"] *dsCLM_end['landfrac']* 1e6).sum(dim=['lat', 'lon']).values
        pools_begin = pools_begin + (dsCLM_begin[pool].fillna(0) * dsCLM_begin["area"] *dsCLM_begin['landfrac']* 1e6).sum(dim=['lat', 'lon']).values

totn_end = (dsCLM_end["NITRATE_TOTN"].fillna(0) * dsCLM_end["area"] *dsCLM_end['landfrac']* 1e6).sum(dim=['lat', 'lon']).values * 1e-12
totn_begin = (dsCLM_begin["NITRATE_TOTN"].fillna(0) * dsCLM_begin["area"] *dsCLM_begin['landfrac']* 1e6).sum(dim=['lat', 'lon']).values * 1e-12

# g to Tg
pools_end = pools_end * 1e-12
pools_begin = pools_begin * 1e-12
nitrate = pools_end - pools_begin
print("nitrate pools at begin:", pools_begin, totn_begin)
print("nitrate pools in the end:", pools_end, totn_end)
print("nitrate pools: ", nitrate, "Tg")

nitrate pools at begin: [0.543787] [0.54378706]
nitrate pools in the end: [0.53708804] [0.53708804]
nitrate pools:  [-0.00669897] Tg


## N Balance

In [16]:
d = fanv2_flux - n2o_nit_total - nox_nit_total - n2o_denit_total - nox_denit_total - n2_total - runoff_fert - runoff_manure - soil_fert - soil_manure - nitrate
print(d)

[0.5434213]
